In [1]:
import random as rnd
import string 
from tqdm import tqdm
import math
import mmh3
import itertools
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Init global parametrs

In [2]:
k = 20
m = 3000
r = 100
n = 1000
N = 0
INF = 1_000_000_000_000_000

In [3]:
def gen_token_name(len:int = 10)-> str:
    """
    This function generate random string with lenght = len.
    """
    return "".join(rnd.choice(string.ascii_letters + string.digits) for _ in range(len))

def gen_address()->int:
    """
    This funciton generate fandom address from diaposon: [1, 1_000_000_000]
    """
    return int(rnd.uniform(1, 1_000_000_000_000))

In [4]:
def hash_address_with_nonce(address: int, number_of_nonce: int):
    global m
    nonce = f'nonce_{number_of_nonce}'
    return mmh3.hash128(str(address) + nonce) % m

def hash_address(address: int)->list:
    global m
    hashed_address = [0]*m

    for i in range(k):
        hashed_address[hash_address_with_nonce(address, i)] = 1
        
    return hashed_address

In [5]:
def find_l(n0)->int:
    """
    This function find l parametr(count of added 1 in mask) from N - size of data base, m, k, and r to get n element by query.
    """
    global N, k, m, r
    return int(math.log(1 - (n0/N)**(1/k))/math.log(1-1/m) - r*k)

In [6]:
class generator:
    def __init__(self, address):
        global r, m
        self.i = 0
        self.j = 1
        self.address = address

    def reset(self):
        self.i = 0
        self.j = 1

    def next(self):
        result = mmh3.hash128(str(self.address[self.i] | self.address[self.j])) % m

        self.j += 1
        if self.j == r:
            self.i += 1;
            self.j = self.i + 1

        if self.j >= r:
            self.reset()

        return result 

class bucket:
    def __init__(self, cnt_address=r):
        self.fully = False
        self.lmin = 0
        self.num = 0
        
        address = []
        for i in range(cnt_address):
            address.append(gen_address())

        self.address = address
        self.generator = generator(address)
    
    def print_info(self):
        info = f'is fully: {self.fully}, lmin: {self.lmin}, num: {self.num}'
        print(info)

In [7]:
class user:
    def __init__(self):
        global r
        self.buckets = [bucket(r)]

        self.cur_bucket = 0
        self.cur_index = 0

    def add_bucket(self):
        global r
        self.buckets.append(bucket(r))

    def generate_bloom(self, number_of_bucket: int, l:int):
        global m, k
        bloom = [0]*m
        for address in self.buckets[number_of_bucket].address:
            for i in range(k):
                bloom[hash_address_with_nonce(address, i)] = 1

        self.buckets[number_of_bucket].generator.reset()
        
        for i in range(int(l)):
            bloom[self.buckets[number_of_bucket].generator.next() % m] = 1

        return bloom
    
    def filter_address(self, number_of_bucket:int, address:list):
        global r
        cnt = 0

        for cur_address in address:
            if cur_address in self.buckets[number_of_bucket].address:
                cnt += 1

        if not self.buckets[number_of_bucket].fully:
          if self.buckets[number_of_bucket].lmin != 0:
              self.buckets[number_of_bucket].lmin = max(self.buckets[number_of_bucket].lmin, find_l(cnt*10))
          else:
              self.buckets[number_of_bucket].lmin = find_l(cnt*10)

        if cnt == r:
          self.buckets[number_of_bucket].fully = True
          self.buckets[number_of_bucket].num = len(address)

        check = False
        if cnt == r and self.cur_bucket > number_of_bucket:
            check = True
        if self.cur_bucket == number_of_bucket and cnt == self.cur_index:
            check = True
        if not check:
            print(f'ERROR! query: [\nbucekt number: {number_of_bucket}\nour address cnt: {cnt}\nreal cur_bucket: {self.cur_bucket}\nreal cur_index: {self.cur_index}\n]\n finished with: {check}\n')
        return cnt
    
    def get_active_address(self):
        global r
        result = self.buckets[self.cur_bucket].address[self.cur_index]

        self.cur_index += 1
        if self.cur_index == r:
            self.cur_bucket += 1
            self.cur_index = 0
            self.add_bucket()

        return result
    

class db_string:
    def __init__(self, address, token):
        self.address = address
        self.hashed_address = hash_address(address=address)
        self.token = token

class server:
    def __init__(self):
        self.data = []
        
    def add_string(self, cur_string: db_string):
        self.data.append(cur_string)

    def get_tokens(self, bloom:list, num:int=INF):
        result = []
        bloom_np = np.array(bloom)
        for cur_str in self.data:
            cur_bloom_np = np.array(cur_str.hashed_address)
            if all(cur_bloom_np == (cur_bloom_np & bloom_np)):
                result.append(cur_str.address)
            if len(result) == num:
                break

        return result

    def get_N(self)->int:
        return len(self.data)

In [8]:
users = []
serv = server()

In [9]:
def addToken(cnt:int=1):
    global users
    global serv
    global N

    for i in range(cnt):
        number_of_user = rnd.randint(0, len(users)-1)
        token = gen_token_name(1)
        address = users[number_of_user].get_active_address()
        cur_str = db_string(address, token)
        serv.add_string(cur_str)
        N = serv.get_N()

def check(number_of_user:int, number_of_bucket:int):
    global INF, n

    cur_l = find_l(n)
    cur_l = max(cur_l, users[number_of_user].buckets[number_of_bucket].lmin)

    bloom = users[number_of_user].generate_bloom(number_of_bucket, cur_l)

    num = INF
    if users[number_of_user].buckets[number_of_bucket].fully:
        num = users[number_of_user].buckets[number_of_bucket].num
            
    address = serv.get_tokens(bloom=bloom, num=num)

    cnt = users[number_of_user].filter_address(number_of_bucket, address)
    
    return [len(address), cnt]

def stress_test(test_cnt, is_random=True, buckets_old=INF, us_id=0, buck_id=0):
    cnts = []

    for i in range(test_cnt):
        user_id = us_id
        bucket_id = buck_id
        if is_random:
          user_id = rnd.randint(0, len(users) - 1)
          bucket_id = rnd.randint(0, min(buckets_old, len(users[user_id].buckets) - 1))

        res = check(user_id, bucket_id)
        cnts.append(res[0])

    if not is_random:
        print('cur size:' + str(res[1]) + ' we get from server: ' + str(res[0]))

    return cnts

def vizualize(cnts:list):
    plt.figure(figsize=(2, 2))
    sns.histplot(cnts, bins=20, kde=True, color='skyblue')

    plt.title('Distribution of count of server ans')
    plt.xlabel('count ans')
    plt.ylabel('Frequency')
    plt.show()

In [10]:
def init_db(user_cnt:int=1000, token_cnt:int=10000):
    global users
    global serv
    global N
    
    N = 0
    users = []
    serv = server()
    
    cout_users = user_cnt
    count_token = token_cnt
    for i in range(cout_users):
        cur_user = user()
        users.append(cur_user)
    addToken(count_token)

In [12]:
init_db(user_cnt=1000, token_cnt=10000)

for i in range(10):
    users[0].buckets[0].print_info()
    stress_test(1, False, us_id=0, buck_id=0)
    addToken(1000)

is fully: False, lmin: 0, num: 0
cur size:10 we get from server: 974
is fully: False, lmin: 2743, num: 0
cur size:10 we get from server: 955
is fully: False, lmin: 2743, num: 0
cur size:13 we get from server: 929
is fully: False, lmin: 2790, num: 0
cur size:15 we get from server: 935
is fully: False, lmin: 2827, num: 0
cur size:16 we get from server: 933
is fully: False, lmin: 2827, num: 0
cur size:16 we get from server: 960
is fully: False, lmin: 2827, num: 0
cur size:18 we get from server: 965
is fully: False, lmin: 2827, num: 0
cur size:19 we get from server: 978
is fully: False, lmin: 2827, num: 0
cur size:21 we get from server: 1004
is fully: False, lmin: 2834, num: 0
cur size:22 we get from server: 978


In [13]:
init_db(user_cnt=1000, token_cnt=10000)

users[0].buckets[0].print_info()
stress_test(1, False, us_id=0, buck_id=0)
addToken(1000)
users[0].buckets[0].print_info()
stress_test(1, False, us_id=0, buck_id=0)
addToken(100000)
users[0].buckets[0].print_info()
stress_test(1, False, us_id=0, buck_id=0)

is fully: False, lmin: 0, num: 0
cur size:7 we get from server: 963
is fully: False, lmin: 2545, num: 0
cur size:10 we get from server: 975
is fully: False, lmin: 2689, num: 0
cur size:100 we get from server: 1014


[1014]